In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import shutil

In [ ]:
dir_train = '/content/drive/MyDrive/Colab/Train_dataset'
dir_mask = '/content/drive/MyDrive/MyProject/Mask_byMajorVoting'
file_train = os.listdir(dir_train)

In [ ]:
dirs_traindata_cropped = [
    '/content/drive/MyDrive/Colab/Train_dataset_cropped_sliding_window_8',
    '/content/drive/MyDrive/Colab/Train_dataset_cropped_sliding_window_4',
    '/content/drive/MyDrive/Colab/Train_dataset_cropped_sliding_window_2'
]
dirs_mask_cropped = [
    '/content/drive/MyDrive/Colab/Mask_dataset_cropped_sliding_window_8',
    '/content/drive/MyDrive/Colab/Mask_dataset_cropped_sliding_window_4',
    '/content/drive/MyDrive/Colab/Mask_dataset_cropped_sliding_window_2'
]

In [ ]:
for i in dirs_traindata_cropped:
    if os.path.isdir(dirs_traindata_cropped[i]) == False:
        os.mkdir(dirs_traindata_cropped[i])
for i in dirs_mask_cropped:
    if os.path.isdir(dirs_mask_cropped[i]) == False:
        os.mkdir(dirs_mask_cropped[i])

In [ ]:
sliding_window = 1
for size in range(3):
    idx = 1
    sliding_window*=2
    for i in range(len(file_train)):
        image = file_train[i]
        mask = file_train[i].replace('.jpg', '_classimg_nonconvex.png')
        if os.path.isfile(f'{dir_train}/{image}') == False or os.path.isfile(f'{dir_mask}/{mask}') == False:
            print(f'pass {dir_train}/{image}')
            continue

        train_img = np.array(Image.open(f'{dir_train}/{image}'))
        mask_img = np.array(Image.open(f'{dir_mask}/{mask}'))

        # Calculate the new dimensions for the cropped image
        new_height = np.shape(train_img)[0] // sliding_window
        new_width = np.shape(train_img)[1] // sliding_window

        # Crop the image
        for h in range(0, np.shape(train_img)[0] - new_height+1, new_height):
            for w in range(0, np.shape(train_img)[1] - new_width+1, new_width):
                train_subimg = train_img[h:h+new_height, w:w+new_width]
                values, counts = np.unique(train_subimg, return_counts=True)
                if np.min(values) >= 205:
                    continue
                ivalue = 1
                value_check = 205
                while True:
                    if value_check not in values:
                        if value_check == 254:
                            print(f'find 205: {file_train[i]}, ({h},{w}) {value_check} ,{ivalue}')
                            ivalue = -1
                        value_check +=ivalue

                    else:
                        break
                percentate = np.sum(counts[np.where(values == value_check)[0][0]:])/(new_height* new_width* 3)
                if percentate >= 0.85:
                    continue

                Image.fromarray(train_subimg).save(f'{dirs_traindata_cropped[size]}/{idx}_{image}')
                mask_subimg = mask_img[h:h+new_height, w:w+new_width]
                Image.fromarray(mask_subimg).save(f'{dirs_mask_cropped[size]}/{idx}_{mask}')
                idx +=1
                print(f'{file_train[i]}, new_height: {new_height}, new_width: {new_width}, idx current: {idx}, sliding: {sliding_window}')
